<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Plotly" data-toc-modified-id="Plotly-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Plotly</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Sankey" data-toc-modified-id="Sankey-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sankey</a></span></li><li><span><a href="#Grouping-by-accounts" data-toc-modified-id="Grouping-by-accounts-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Grouping by accounts</a></span><ul class="toc-item"><li><span><a href="#Many-to-many" data-toc-modified-id="Many-to-many-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Many-to-many</a></span></li><li><span><a href="#Saint-Petersburg-May-billable" data-toc-modified-id="Saint-Petersburg-May-billable-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Saint-Petersburg May billable</a></span></li></ul></li></ul></div>

In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import pandas.io.sql as psql
import datetime
from dateutil.relativedelta import relativedelta
import time
import sys
sys.path.append('../src')
from datahub import PostgreSQLBase, ToDatahubWriter
import seaborn as sns
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(name)s : %(levelname)s : %(message)s')

consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(formatter)
logger.addHandler(consoleHandler)

import auxiliary as aux
reload (aux)

postgres = PostgreSQLBase()
df_proc = aux.DataFrameProcessor()
loc_ids = aux.City_Country()

In [1]:
import plotly.plotly as py
import plotly
plotly.__version__

u'3.10.0'

# Plotly

In [19]:
# !pip install plotly --upgrade

  Stored in directory: C:\Users\Alexey_Yakunin\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [2]:
import plotly.plotly as py

data = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label = ["A1", "A2", "B1", "B2", "C1", "C2", "C3"],
      color = ["blue", "blue", "blue", "blue", "blue", "blue", "red"]
    ),
    link = dict(
      source = [0,1,0,2,3,3,2],
      target = [2,3,3,4,4,5,6],
      value = [8,4,2,6,4,2,2]
  ))

layout =  dict(
    title = "Basic Sankey Diagram",
    font = dict(
      size = 10
    )
)

fig = dict(data=[data], layout=layout)
py.iplot(fig, validate=False)



# Data

In [6]:
snaps = ['2019-02-01', '2019-02-07', '2019-02-14',
         '2019-03-01', '2019-03-07', '2019-03-14',
         '2019-04-01', '2019-04-07', '2019-04-14',
         '2019-05-01', '2019-05-07', '2019-05-14',
         '2019-06-01', '2019-06-07', '2019-06-14', '2019-06-21']

In [21]:
sub_snaps = [s for s in snaps if int(s[6]) >= int(max_date[6])]

In [25]:
[s for s in sub_snaps if int(s[6]) == int(max_date[6])+1
] + [s for s in sub_snaps if (s[8:] == '01')&(int(s[6]) > int(max_date[6])+1)] + snaps[-1:]

['2019-04-01',
 '2019-04-07',
 '2019-04-14',
 '2019-05-01',
 '2019-06-01',
 '2019-06-21']

In [27]:
%%time
d = datetime.datetime.strptime('2019-01-01', '%Y-%m-%d')

for i in range(5):
    
    min_date = datetime.datetime.strftime(d + relativedelta(months=i), '%Y-%m-%d')
    max_date = datetime.datetime.strftime(d + relativedelta(months=i+1), '%Y-%m-%d')
    
    sub_snaps = [s for s in snaps if int(s[6]) >= int(max_date[6])]
    days = [s for s in sub_snaps if int(s[6]) == int(max_date[6])
        ] + [s for s in sub_snaps if (s[8:] == '01')&(int(s[6]) >= int(max_date[6])+1)] + snaps[-1:]
    
    params = (min_date, max_date, list(days))
    
    logger.info('{0}, {1}, {2}'.format(*params))

    df = postgres.get_data('time_reporting_changes.sql', param=params, param_type='tuple')
    
    df.to_pickle('../data/time_reporting_changes{}.pkl'.format(min_date))

2019-06-21 20:48:35,744 - root : INFO : 2019-01-01, 2019-02-01, ['2019-02-01', '2019-02-07', '2019-02-14', '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01', '2019-06-21']
2019-06-21 21:37:07,286 - root : INFO : 2019-02-01, 2019-03-01, ['2019-03-01', '2019-03-07', '2019-03-14', '2019-04-01', '2019-05-01', '2019-06-01', '2019-06-21']
2019-06-21 22:02:01,917 - root : INFO : 2019-03-01, 2019-04-01, ['2019-04-01', '2019-04-07', '2019-04-14', '2019-05-01', '2019-06-01', '2019-06-21']
2019-06-21 22:21:47,660 - root : INFO : 2019-04-01, 2019-05-01, ['2019-05-01', '2019-05-07', '2019-05-14', '2019-06-01', '2019-06-21']
2019-06-21 22:35:36,151 - root : INFO : 2019-05-01, 2019-06-01, ['2019-06-01', '2019-06-07', '2019-06-14', '2019-06-21', '2019-06-21']


Wall time: 2h 16s


In [ ]:
df.shape

In [ ]:
df = pd.read_pickle('../data/time_reporting_changes.pkl')

# Sankey

In [3]:
def time_report_moving(month='Jan', corr='yes'):
    
    def regular(x):

        d1 = x['from'][:10]
        d2 = x['to'][:10]
        b = x['from'][11:]

        if b == 'billable':
            s = df[
                (df.date.astype(str) == d1) &
                (df.billable == True) &
                (df.id.isin(
                    df[
                        (df.date.astype(str) == d2) &
                        (df.billable == True)
                    ].id 
                ))
            ].duration.sum()

        else:
            s = df[
                (df.date.astype(str) == d1) &
                (df.billable == False) &
                (df.id.isin(
                    df[
                        (df.date.astype(str) == d2) &
                        (df.billable == False)
                    ].id 
                ))
            ].duration.sum()

        return s

    def correct(x):

        b = x['from'][11:]
        v = x['val']
        d_min = min(df.date.unique().astype(str))
        d_max = max(df.date.unique().astype(str))

        if b == 'billable':

            corr = df[
                (df.date.astype(str) == d_min) &
                (df.billable == True) &
                (df.id.isin(
                    df[
                        (df.date.astype(str) == d_max) &
                        (df.billable == True)
                    ].id 
                ))
            ].duration.sum()

        else:

            corr = df[
                (df.date.astype(str) == d_min) &
                (df.billable == False) &
                (df.id.isin(
                    df[
                        (df.date.astype(str) == d_max) &
                        (df.billable == False)
                    ].id 
                ))
            ].duration.sum()

        return (v-corr)

    def to_cancel(x):

        d1 = x['from'][:10]
        d2 = min([d for d in df.date.unique().astype(str) if d > d1])
        b = x['from'][11:]

        if b == 'billable':
            s = df[
                (df.date.astype(str) == d1) &
                (df.billable == True) &
                (~df.id.isin(
                    df[
                        (df.date.astype(str) == d2) &
                        (df.billable == True)
                    ].id 
                ))
            ].duration.sum()

        else:
            s = df[
                (df.date.astype(str) == d1) &
                (df.billable == False) &
                (~df.id.isin(
                    df[
                        (df.date.astype(str) == d2) &
                        (df.billable == False)
                    ].id 
                ))
            ].duration.sum()

        return s

    def from_new(x):

        d2 = x['to'][:10]
        d1 = max([d for d in df.date.unique().astype(str) if d < d2])
        b = x['to'][11:]

        if b == 'billable':
            s = df[
                (df.date.astype(str) == d2) &
                (df.billable == True) &
                (~df.id.isin(
                    df[
                        (df.date.astype(str) == d1) &
                        (df.billable == True)
                    ].id 
                ))
            ].duration.sum()

        else:
            s = df[
                (df.date.astype(str) == d2) &
                (df.billable == False) &
                (~df.id.isin(
                    df[
                        (df.date.astype(str) == d1) &
                        (df.billable == False)
                    ].id 
                ))
            ].duration.sum()

        return s
    
    
    months = {
        'Jan' : '2019-01-01',
        'Feb' : '2019-02-01',
        'Mar' : '2019-03-01',
        'Apr' : '2019-04-01',
        'May' : '2019-05-01'
    }
    
    df = pd.read_pickle('../data/time_reporting_changes{}.pkl'.format(months[month]))
    
    lables = ['New hours', 'Cancelled hours']
    for d in df.date.unique().astype(str):
        for b in ['billable', 'NONbillable']:
            lables.append(d + '_' + b)
            
    colors = ['blue', 'red'] + ['blue' if l[-11:] == 'NONbillable' else 'green' for l in lables][2:]
    
    moves = []
    for d1 in list(df.date.unique().astype(str))[:-1]:
        d2 = min([d for d in df.date.unique().astype(str) if d > d1])
        for b in ['billable', 'NONbillable']:
            moves.append([d1 + '_' + b, d2 + '_' + b])
    moves = pd.DataFrame(moves, columns=['from', 'to'])
    
    moves_from = moves.copy()
    moves_from['from'] = 'New hours'
    moves_to = moves.copy()
    moves_to['to'] = 'Cancelled hours'

    moves['val'] = moves.apply(regular, axis=1)
    
    if corr == 'yes':
        moves['val'] = moves.apply(correct, axis=1)
        
    moves_from['val'] = moves_from.apply(from_new, axis=1)
    moves_to['val'] = moves_to.apply(to_cancel, axis=1)

    moves = pd.concat([moves, moves_from, moves_to])

    moves['from_id'] = moves['from'].map(lambda x: [i for i,l in enumerate(lables) if l == x][0]).astype(int)
    moves['to_id'] = moves['to'].map(lambda x: [i for i,l in enumerate(lables) if l == x][0]).astype(int)

    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = lables,
          color = colors
        ),
        link = dict(
          source = list(moves['from_id']),
          target = list(moves['to_id']),
          value = list(moves['val'])
      ))

    layout =  dict(
        title = "{} hours".format(month),
        font = dict(
          size = 10
        )
    )

    fig = dict(data=[data], layout=layout)
    
    plotly.offline.init_notebook_mode(connected=True)
    
    display(py.iplot(fig, validate=False))
    
    
    moves.to_pickle('../data/time_reporting_final_{}.pkl'.format(months[month]))

In [6]:
%%time
time_report_moving(month='Jan', corr='yes')

2019-06-23 08:41:39,740 - urllib3.connectionpool : DEBUG : Starting new HTTPS connection (1): plot.ly:443
2019-06-23 08:41:41,635 - urllib3.connectionpool : DEBUG : https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


Wall time: 11min 1s


In [7]:
%%time
time_report_moving(month='Feb', corr='yes')

2019-06-23 08:49:47,904 - urllib3.connectionpool : DEBUG : Starting new HTTPS connection (1): plot.ly:443
2019-06-23 08:49:49,479 - urllib3.connectionpool : DEBUG : https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


Wall time: 8min 8s


In [8]:
%%time
time_report_moving(month='Mar', corr='yes')

2019-06-23 08:55:52,862 - urllib3.connectionpool : DEBUG : Starting new HTTPS connection (1): plot.ly:443
2019-06-23 08:55:54,266 - urllib3.connectionpool : DEBUG : https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


Wall time: 6min 3s


In [9]:
%%time
time_report_moving(month='Apr', corr='yes')

2019-06-23 09:00:34,497 - urllib3.connectionpool : DEBUG : Starting new HTTPS connection (1): plot.ly:443
2019-06-23 09:00:35,936 - urllib3.connectionpool : DEBUG : https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


Wall time: 4min 40s


In [10]:
%%time
time_report_moving(month='May', corr='yes')

2019-06-23 09:04:04,301 - urllib3.connectionpool : DEBUG : Starting new HTTPS connection (1): plot.ly:443
2019-06-23 09:04:06,144 - urllib3.connectionpool : DEBUG : https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


Wall time: 3min 29s


In [11]:
%%time
time_report_moving(month='Jan', corr='no')

2019-06-23 09:12:15,822 - urllib3.connectionpool : DEBUG : Starting new HTTPS connection (1): plot.ly:443
2019-06-23 09:12:17,403 - urllib3.connectionpool : DEBUG : https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


Wall time: 8min 16s


In [12]:
%%time
time_report_moving(month='Feb', corr='no')

2019-06-23 09:18:34,186 - urllib3.connectionpool : DEBUG : Starting new HTTPS connection (1): plot.ly:443
2019-06-23 09:18:36,595 - urllib3.connectionpool : DEBUG : https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


Wall time: 6min 18s


In [13]:
%%time
time_report_moving(month='Mar', corr='no')

2019-06-23 09:23:27,654 - urllib3.connectionpool : DEBUG : Starting new HTTPS connection (1): plot.ly:443
2019-06-23 09:23:29,098 - urllib3.connectionpool : DEBUG : https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


Wall time: 4min 50s


In [14]:
%%time
time_report_moving(month='Apr', corr='no')

2019-06-23 09:27:13,364 - urllib3.connectionpool : DEBUG : Starting new HTTPS connection (1): plot.ly:443
2019-06-23 09:27:14,996 - urllib3.connectionpool : DEBUG : https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


Wall time: 3min 44s


In [15]:
%%time
time_report_moving(month='May', corr='no')

2019-06-23 09:30:03,644 - urllib3.connectionpool : DEBUG : Starting new HTTPS connection (1): plot.ly:443
2019-06-23 09:30:05,635 - urllib3.connectionpool : DEBUG : https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


Wall time: 2min 49s


# Grouping by accounts

In [5]:
def grouping_by_accounts(month = 'Jan'):
    
    logger.info('Script for {} was started'.format(month))
    
    months = {
        'Jan' : '2019-01-01',
        'Feb' : '2019-02-01',
        'Mar' : '2019-03-01',
        'Apr' : '2019-04-01',
        'May' : '2019-05-01'
    }
    
    df = pd.read_pickle('../data/time_reporting_changes{}.pkl'.format(months[month]))
    
    logger.info(df.shape)

    proj = set(df.project_uid_projection.unique()) - set(['npa', 'vac'])

    proj = postgres.get_data('bind_project_by_ids.sql', param=list(
        proj), param_type='list')

    pos = list(df.bind_position_id.dropna().astype(int).unique())

    pos = postgres.get_data('bind_projects_by_positions.sql', param=pos, param_type='list')

    pos.position_id = pos.position_id.astype(str)

    df = pd.merge(
        df, pos.set_index('position_id'), how='left',
        left_on = 'bind_position_id', right_index = True
    )

    proj.external_id = proj.external_id.astype(str)

    df = pd.merge(
        df, proj.set_index('external_id'), how='left',
        left_on = 'project_uid_projection', right_index = True
    )
    
    logger.info(df.shape)

    df['bucket'] = np.where(df.project_uid_projection == 'npa', 'NPA', 
                            np.where(df.project_uid_projection == 'vac', 'VAC', df.account)
                           )
    df['billing_bucket'] = np.where(
        df.project_uid_projection == 'vac', 'Acc_vac', np.where(
            df.billable == True, 'Acc_bill', np.where(
                (df.project_uid_projection == 'npa')|(df.billing_type_name == 'On Training'), 'Acc_npa', np.where(
                    (df.account == 'EPM') & (df.billing_type_name == 'Temporary'), 'EPM_npa', 'Acc_nonbill'
                )
            )
        )
    )


    df['key'] = df.bucket.astype(str) + '--' + df.billable.astype(str) + '--' + df.billing_bucket

    df1 = df[df.date == min(df.date.unique())]
    df1 = pd.DataFrame(df1.groupby('key').duration.sum()).rename(columns={'duration' : min(df.date.unique().astype(str))})

    for d in list(df.date.unique().astype(str))[1:]:

        df2 = df[df.date.astype(str) == d]

        df2 = pd.DataFrame(df2.groupby('key').duration.sum()).rename(columns={'duration' : d})

        df1 = pd.merge(
            df1, df2, how='outer',
            left_index = True, right_index = True
        )


    df1 = df1.reset_index()
    df1['account'] = df1['key'].map(lambda k: k.split('--')[0])
    df1['billable'] = df1['key'].map(lambda k: k.split('--')[1])
    df1['billing_bucket'] = df1['key'].map(lambda k: k.split('--')[2])


    df1.to_excel('../data/reported_time_by_accounts_{}.xlsx'.format(month))
    
    logger.info('data were loaded to file: reported_time_by_accounts_{}.xlsx'.format(month))

In [6]:
%%time
grouping_by_accounts(month = 'Jan')

2019-06-26 11:09:52,786 - root : INFO : Script for Jan was started
2019-06-26 11:11:40,982 - root : INFO : (7139381, 13)
2019-06-26 11:12:11,651 - root : INFO : (7139381, 16)
2019-06-26 11:13:04,459 - root : INFO : data were loaded to file: reported_time_by_accounts_Jan.xlsx


Wall time: 3min 25s


In [7]:
%%time
grouping_by_accounts(month = 'Feb')

2019-06-26 11:13:18,539 - root : INFO : Script for Feb was started
2019-06-26 11:14:34,688 - root : INFO : (6248739, 13)
2019-06-26 11:14:53,289 - root : INFO : (6248739, 16)
2019-06-26 11:15:32,790 - root : INFO : data were loaded to file: reported_time_by_accounts_Feb.xlsx


Wall time: 2min 21s


In [8]:
%%time
grouping_by_accounts(month = 'Mar')

2019-06-26 11:15:39,663 - root : INFO : Script for Mar was started
2019-06-26 11:16:42,928 - root : INFO : (5506903, 13)
2019-06-26 11:16:57,530 - root : INFO : (5506903, 16)
2019-06-26 11:17:28,703 - root : INFO : data were loaded to file: reported_time_by_accounts_Mar.xlsx


Wall time: 1min 53s


In [9]:
%%time
grouping_by_accounts(month = 'Apr')

2019-06-26 11:17:33,714 - root : INFO : Script for Apr was started
2019-06-26 11:18:37,697 - root : INFO : (4972741, 13)
2019-06-26 11:18:51,342 - root : INFO : (4972741, 16)
2019-06-26 11:19:17,315 - root : INFO : data were loaded to file: reported_time_by_accounts_Apr.xlsx


Wall time: 1min 47s


In [10]:
%%time
grouping_by_accounts(month = 'May')

2019-06-26 11:19:20,867 - root : INFO : Script for May was started
2019-06-26 11:20:17,536 - root : INFO : (4644177, 13)
2019-06-26 11:20:30,444 - root : INFO : (4644177, 16)
2019-06-26 11:20:51,829 - root : INFO : data were loaded to file: reported_time_by_accounts_May.xlsx


Wall time: 1min 33s


## Many-to-many

In [75]:
df['key'] = df.pmc_user_id.astype(str) + df.timesheet_date.astype(str)
df['grouper'] = df.pmc_user_id.astype(str) + df.timesheet_date.astype(str) + df.billable.astype(str) \
                + df.bucket.astype(str)

In [79]:
pd.DataFrame(df.drop_duplicates('grouper').groupby('key')['grouper'].count()).sort_values('grouper', ascending=False)

,grouper
key,
40607414003014296952019-01-24,8
40607414003014296952019-01-25,8
40607414003014296952019-01-23,7
40607414003014296952019-01-21,7
40607414003014296872019-01-28,7
40607414003014296872019-01-17,7
40607414003014297212019-01-15,7
40607414003014297212019-01-08,7
40607414003014296872019-01-16,7


In [89]:
df[
    (df.date.astype(str) == '2019-02-01') &
    (df.pmc_user_id == '4060741400301429695') &
    (df.timesheet_date.astype(str) == '2019-01-24')
][['id', 'bucket', 'billable', 'duration']]

,id,bucket,billable,duration
618330,60735589,NPA,False,0.25
618477,60894658,HEMA,True,1.50
642898,60766039,ADN,False,5.00
649449,60742759,ADN,False,0.50
649533,60742880,ADN,False,0.75


In [88]:
df[
    (df.date.astype(str) == '2019-06-21') &
    (df.pmc_user_id == '4060741400301429695') &
    (df.timesheet_date.astype(str) == '2019-01-24')
][['id', 'bucket', 'billable', 'duration']]

,id,bucket,billable,duration
6872906,62477199,PLU,True,3.20
6872907,62476733,HEMA,False,1.60
6872908,62270205,ADN,True,1.60
6873023,64840508,MDB,True,1.60
6898605,60766039,ADN,False,5.00
6905156,60742759,ADN,False,0.50
6905240,60742880,ADN,False,0.75


## Saint-Petersburg May billable

In [38]:
df = pd.read_pickle('../data/time_reporting_changes2019-05-01.pkl')

In [39]:
logger.info(df.shape)

proj = set(df.project_uid_projection.unique()) - set(['npa', 'vac'])

proj = postgres.get_data('bind_project_by_ids.sql', param=list(
    proj), param_type='list')

pos = list(df.bind_position_id.dropna().astype(int).unique())

pos = postgres.get_data('bind_projects_by_positions.sql', param=pos, param_type='list')

pos.position_id = pos.position_id.astype(str)

df = pd.merge(
    df, pos.set_index('position_id'), how='left',
    left_on = 'bind_position_id', right_index = True
)

proj.external_id = proj.external_id.astype(str)

df = pd.merge(
    df, proj.set_index('external_id'), how='left',
    left_on = 'project_uid_projection', right_index = True
)

logger.info(df.shape)

df['bucket'] = np.where(df.project_uid_projection == 'npa', 'NPA', 
                        np.where(df.project_uid_projection == 'vac', 'VAC', df.account)
                       )
df['billing_bucket'] = np.where(
    df.project_uid_projection == 'vac', 'Acc_vac', np.where(
        df.billable == True, 'Acc_bill', np.where(
            (df.project_uid_projection == 'npa')|(df.billing_type_name == 'On Training'), 'Acc_npa', np.where(
                (df.account == 'EPM') & (df.billing_type_name == 'Temporary'), 'EPM_npa', 'Acc_nonbill'
            )
        )
    )
)

2019-06-26 11:42:20,650 - root : INFO : (4644177, 13)
2019-06-26 11:42:37,661 - root : INFO : (4644177, 16)


In [40]:
emp = postgres.get_data('bind_locations_by_employees.sql')

In [41]:
emp.employee_id = emp.employee_id.astype(str)
emp = emp[emp.employee_id.isin(df.pmc_user_id.unique().astype(str))]
emp

,employee_id,location_name,full_name
3,4060741400371153951,Hong Kong SAR,Travis Yeung
4,4000741400003261333,"Mountain View, CA",Aliaksei Trushchankou
5,4060741400005328490,"Mountain View, CA",Oleksandr Gorban
6,4060741400009503670,Kharkiv,Artem Starushko
9,4000741400007699437,"Mountain View, CA",Vladimir Sauta
11,4060741400034494457,"Mountain View, CA",Artem Zhuravsky
13,4060741400039650819,Kyiv,Dmytro Budnichenko
14,4060741400006053364,"Mountain View, CA",Dmitry Kravtsov
16,4060741400040840281,"Mountain View, CA",Aleksei Egorov
17,4060741400336284121,"Boston, MA",Alexander Broerman


In [42]:
df = pd.merge(
    df, emp.set_index('employee_id'), how='left',
    left_on = 'pmc_user_id', right_index = True
)

In [43]:
df.shape

(4644177, 20)

In [44]:
df = df[
    (df.location_name == 'Saint-Petersburg')
#     & (df.billing_bucket == 'Acc_bill')
]

In [47]:
emp = df[
    (df.date.astype(str) == '2019-06-01') &
    (~df.id.isin(
        df[df.date.astype(str) == '2019-06-21'].id.unique()
    ))
 ]#[['timesheet_date', 'activity_name', 'duration', 'bind_position_id', 'full_name', 'container', 'account']].to_excel(
#'../data/Saint-Petersburg_May_changed_billable_reported_hours_per_empl.xlsx')

In [51]:
df[
    (df.date.astype(str) == '2019-06-01') &
    (~df.id.isin(
        df[df.date.astype(str) == '2019-06-21'].id.unique()
    ))
].duration.sum()

3312.5500000000002

In [45]:
df['key'] = df.bucket.astype(str) + '--' + df.billable.astype(str) + '--' + df.billing_bucket

df1 = df[df.date == min(df.date.unique())]
df1 = pd.DataFrame(df1.groupby('key').duration.sum()).rename(columns={'duration' : min(df.date.unique().astype(str))})

for d in list(df.date.unique().astype(str))[1:]:

    df2 = df[df.date.astype(str) == d]

    df2 = pd.DataFrame(df2.groupby('key').duration.sum()).rename(columns={'duration' : d})

    df1 = pd.merge(
        df1, df2, how='outer',
        left_index = True, right_index = True
    )


df1 = df1.reset_index()
df1['account'] = df1['key'].map(lambda k: k.split('--')[0])
df1['billable'] = df1['key'].map(lambda k: k.split('--')[1])
df1['billing_bucket'] = df1['key'].map(lambda k: k.split('--')[2])

In [46]:
df1.to_excel('../data/Saint-Petersburg_May_changed_billable_reported_hours_per_accounts.xlsx')